In [1]:
!pip install transformers datasets torch scikit-learn gradio


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/59.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/59.6 MB ? eta -:--:--
   ---------------------------------------- 0.3/59.6 MB ? eta -:--:--
    --------------------------------------- 0.8/59.6 MB 1.4 MB/s eta 0:00:43
    --------------------------------------- 1.0/59.6 MB 1.5 MB/s eta 0:00:40
    --------------------------------------- 1.3/59.6 MB 1.5 MB/s eta 0:00:40
   - -------------------------------------- 1.6/59.6 MB 1.4 MB/s eta 0:00:41
   - -------------------------------------- 1.8/59.6 MB 1.4 MB/s eta 0:00:41
   - -------------------------------------- 2.1/59.6 MB 1.4 MB/s eta 0:00:41
   - -------------------------------------- 2.6/59.6 MB 1.5 MB/s eta 0:00:40
   - -------------------------------------- 2.9/59.6 MB 1.4 MB/s eta 0:00:40
   -- ------------------------------------- 3.1/59.6 MB 1.4 MB/s eta 0:00:40
   -- -------------

In [2]:
from datasets import load_dataset

# Load AG News dataset from Hugging Face
dataset = load_dataset("ag_news")

# See a sample
print(dataset["train"][0])


README.md: 0.00B [00:00, ?B/s]

C:\Users\R Y Z E N\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\R Y Z E N\.cache\huggingface\hub\datasets--ag_news. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

{'text': "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.", 'label': 2}


In [3]:
from transformers import AutoTokenizer

# Load BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=128)

# Apply tokenization
tokenized_dataset = dataset.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\R Y Z E N\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\R Y Z E N\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [4]:
# Remove unnecessary column
tokenized_dataset = tokenized_dataset.remove_columns(["text"])

# Rename label column to 'labels' as expected by Hugging Face
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

# Set format for PyTorch
tokenized_dataset.set_format("torch")


In [5]:
from transformers import AutoModelForSequenceClassification

# Load BERT with a classification head (4 output classes)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score

# Define evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted")
    }

# Training configuration
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)


In [2]:
import transformers
print(transformers.__version__)

4.52.4


In [4]:
import torch
print(torch.cuda.is_available())

False


In [ ]:
# Get accuracy and F1
eval_results = trainer.evaluate()
print(eval_results)
